In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import random

In [2]:
df = pd.read_excel("actives.xlsx")

cleanup_types = [
    'deck_1',
    'kitchen',
    'deck_0',
    'deck_brush',
    'bathroom_2',
    'bathroom_3',
    'stairs',
]

for c in cleanup_types:
    df[c] = 0

df.to_excel("actives.xlsx", index=False)
df.head()

,name,inhouse,deck_1,kitchen,deck_0,deck_brush,bathroom_2,bathroom_3,stairs
0,Andre,2.0,0,0,0,0,0,0,0
1,Austin,2.0,0,0,0,0,0,0,0
2,Bhuvan,2.0,0,0,0,0,0,0,0
3,Christian,3.0,0,0,0,0,0,0,0
4,Clement,3.0,0,0,0,0,0,0,0


In [4]:
df['name']

0         Andre
1        Austin
2        Bhuvan
3     Christian
4       Clement
5         David
6        Delwin
7         Dylan
8           Ian
9         Jacob
10          Jon
11        Madhu
12     Marshall
13        Marty
14        Mateo
15       Naveen
16       Neehal
17     Roderick
18        Ronal
19          Ron
20        Ronan
21    Siddharth
22        Simon
23        Toney
24        Vivek
25         Yash
26       Yashas
Name: name, dtype: object

In [3]:
num_people = len(df)
# Example input
start_date_str = "2026-01-15"  # YYYY-MM-DD
end_date_str = "2026-05-07"

# Convert to datetime objects
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

# Compute total days
total_days = (end_date - start_date).days + 1  # +1 to include start date

# Compute number of weeks (round up for partial weeks)
num_weeks = math.ceil(total_days / 7)

print(f"Total weeks: {num_weeks}")

# minimum people required per cleanup per week
min_per_week = {
    "deck_1": 1,
    "deck_0": 3,
    "kitchen": 5,
    "stairs": 2,
    "deck_brush": 2,
    "bathroom_2": 2,
    "bathroom_3": 2,
}

# compute extra people per week
extra_per_week = num_people - sum(min_per_week.values())

# ordered cleanups for extra assignment
cleanup_order = list(min_per_week.keys())

# actual people per cleanup per week including extras
per_week_actual = min_per_week.copy()
for i in range(extra_per_week):
    per_week_actual[cleanup_order[i % len(cleanup_order)]] += 1

# now calculate total per-person cleanups over the semester
exact = {k: (v * num_weeks) / num_people for k, v in per_week_actual.items()}

# base (floor) for each cleanup
base = {k: int(exact[k]) for k in exact}

# remaining cleanups to reach exactly num_weeks per person
missing = num_weeks - sum(base.values())

# distribute missing cleanups to the largest fractional parts
fractions = sorted(
    exact.keys(),
    key=lambda k: exact[k] - base[k],
    reverse=True
)
for k in fractions[:missing]:
    base[k] += 1

print(f"Theoretical per-person cleanup target for {num_weeks} weeks...")
for k, v in base.items():
    print(f"{k}: {v}")


Total weeks: 17
Theoretical per-person cleanup target for 17 weeks...
deck_1: 2
deck_0: 3
kitchen: 4
stairs: 2
deck_brush: 2
bathroom_2: 2
bathroom_3: 2


In [4]:
per_week_actual

{'deck_1': 3,
 'deck_0': 5,
 'kitchen': 6,
 'stairs': 3,
 'deck_brush': 3,
 'bathroom_2': 3,
 'bathroom_3': 3}

In [5]:
import json

output = {
    "num_people": num_people,
    "num_weeks": num_weeks,
    "cleanup_types": cleanup_types,
    "min_per_week": min_per_week,
    "per_week_actual": per_week_actual,
    "theoretical_per_person": base,
}

with open("cleanup_config.json", "w") as f:
    json.dump(output, f, indent=4)

print("Saved cleanup_config.json")

Saved cleanup_config.json


In [6]:
import random
import math
from collections import defaultdict

# persistent state across weeks
assigned_so_far = {
    name: defaultdict(int) for name in df['name']
}

last_cleanup = {
    name: None for name in df['name']
}

for c in cleanup_types:
    df[c] = df[c].fillna(0)

# Similarly for assigned_so_far
names = df["names"].tolist()
for person in names:
    for c in cleanup_types:
        if math.isnan(assigned_so_far[person][c]):
            assigned_so_far[person][c] = 0


In [7]:
def schedule_one_week_final(
    week,
    df,
    cleanup_types,
    per_week_actual,
    base,
    assigned_so_far,
    last_cleanup,
    num_weeks
):

    names = list(df['names'])
    random.shuffle(names)

    week_assignment = {}
    used_people = set()

    # Coverage milestone: ensure everyone does each cleanup at least once by week 10
    coverage_target = 1 if week <= 10 else None

    # Compute per-person deficit and available cleanups
    person_deficit = {}
    person_remaining = {}
    for person in names:
        person_deficit[person] = {}
        remaining_count = 0
        for c in cleanup_types:
            person_deficit[person][c] = base[c] - assigned_so_far[person][c]
            if assigned_so_far[person][c] < base[c] + 1:
                remaining_count += 1
        person_remaining[person] = remaining_count

    # Shuffle cleanups each week
    ordered_cleanups = cleanup_types.copy()
    random.shuffle(ordered_cleanups)

    for cleanup in ordered_cleanups:
        slots = per_week_actual[cleanup]

        # Build list of eligible candidates
        candidates = []
        for person in names:
            if person in used_people:
                continue
            # Skip if person already hit base+1 for this cleanup
            if assigned_so_far[person][cleanup] >= base[cleanup] + 1:
                continue
            # Skip person if they did this cleanup last week for kitchen/deck_0
            if cleanup in {"kitchen", "deck_0"} and last_cleanup[person] == cleanup:
                continue

            # Coverage requirement
            if coverage_target is not None and assigned_so_far[person][cleanup] < coverage_target:
                candidates.append((float('inf'), -person_remaining[person], random.random(), person))
                continue

            # Deficit-aware assignment
            deficit = person_deficit[person][cleanup]
            # Prioritize deficits > 0 first
            priority = deficit
            candidates.append((priority, -person_remaining[person], random.random(), person))

        # Sort candidates:
        # 1. highest deficit (people below target)
        # 2. least remaining eligible cleanups
        # 3. random tiebreak
        candidates.sort(reverse=True)

        # Edge case: not enough candidates, allow last-resort people
        if len(candidates) < slots:
            for person in names:
                if person in used_people:
                    continue
                if assigned_so_far[person][cleanup] < base[cleanup] + 1 and person not in [p for *_, p in candidates]:
                    deficit = person_deficit[person][cleanup]
                    candidates.append((deficit, -person_remaining[person], random.random(), person))
            candidates.sort(reverse=True)

        selected = [p for *_, p in candidates[:slots]]

        for person in selected:
            week_assignment[person] = cleanup
            used_people.add(person)

    # Sanity check
    if len(week_assignment) != len(names):
        raise RuntimeError(f"Week {week}: incomplete assignment")

    # Update state
    for person, cleanup in week_assignment.items():
        assigned_so_far[person][cleanup] += 1
        last_cleanup[person] = cleanup
        df.loc[df['names'] == person, cleanup] += 1

    return week_assignment


In [12]:
for week in range(1, num_weeks + 1):
    schedule_one_week_final(
        week,
        df,
        cleanup_types,
        per_week_actual,
        base,
        assigned_so_far,
        last_cleanup,
        num_weeks
    )

# Check ±1 global fairness
for name in df['names']:
    for c in cleanup_types:
        assert abs(assigned_so_far[name][c] - base[c]) <= 1


In [13]:
# Print a summary table of assigned cleanups per person
import pandas as pd

summary = pd.DataFrame.from_dict(assigned_so_far, orient='index')
summary = summary[cleanup_types]  # keep only cleanup columns
summary['total'] = summary.sum(axis=1)

print("Assigned cleanups per person:\n")
print(summary)

# Optionally, also see deviation from base
deviation = summary.copy()
for c in cleanup_types:
    deviation[c] = deviation[c] - base[c]

print("\nDeviation from theoretical base (base ±1 expected):\n")
print(deviation)


Assigned cleanups per person:

           deck_1  kitchen  deck_0  deck_brush  bathroom_2  bathroom_3  \
Andre           2        4       3           2           2           2   
Austin          2        4       4           2           2           2   
Bhuvan          2        4       4           2           1           2   
Christian       2        3       3           3           2           2   
Clement         2        4       3           2           2           2   
David           1        4       3           2           3           2   
Delwin          2        4       3           2           2           2   
Dylan           2        3       3           3           2           2   
Ian             2        4       3           1           3           2   
Jacob           2        5       3           2           1           2   
Jon             2        5       3           2           1           2   
Madhu           2        4       3           2           2           2   
Marty  

In [8]:
weekly_assignments = {}  # key = week number, value = week_assignment dict

for week in range(1, num_weeks + 1):
    week_assignment = schedule_one_week_final(
        week,
        df,
        cleanup_types,
        per_week_actual,
        base,
        assigned_so_far,
        last_cleanup,
        num_weeks
    )
    weekly_assignments[week] = week_assignment


In [9]:
import pandas as pd

weekly_summary = []

for week, assignment in weekly_assignments.items():
    week_df = pd.DataFrame.from_dict(assignment, orient='index', columns=['cleanup'])
    week_df.reset_index(inplace=True)
    week_df.rename(columns={'index': 'name'}, inplace=True)
    week_df['week'] = week
    weekly_summary.append(week_df)

weekly_summary_df = pd.concat(weekly_summary, ignore_index=True)

In [10]:
pivot_df = weekly_summary_df.pivot(index='name', columns='week', values='cleanup')
pivot_df

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
name,,,,,,,,,,,,,,,,,
Andre,deck_0,stairs,kitchen,bathroom_3,deck_brush,deck_1,kitchen,bathroom_2,deck_brush,deck_0,kitchen,deck_0,deck_1,deck_brush,bathroom_3,stairs,deck_0
Austin,deck_1,kitchen,stairs,deck_0,deck_0,kitchen,bathroom_3,deck_brush,bathroom_2,stairs,deck_0,deck_brush,bathroom_2,kitchen,deck_0,kitchen,deck_1
Bhuvan,deck_1,bathroom_2,deck_0,kitchen,deck_brush,kitchen,bathroom_3,stairs,deck_0,stairs,bathroom_2,deck_1,deck_brush,kitchen,deck_0,kitchen,bathroom_3
Christian,deck_brush,kitchen,stairs,bathroom_3,deck_0,bathroom_2,kitchen,deck_1,kitchen,deck_0,bathroom_3,stairs,bathroom_2,deck_0,kitchen,deck_brush,kitchen
Clement,bathroom_3,deck_1,kitchen,deck_brush,kitchen,deck_0,stairs,deck_0,deck_brush,kitchen,bathroom_2,kitchen,stairs,bathroom_3,bathroom_2,deck_0,kitchen
David,deck_1,bathroom_3,kitchen,deck_brush,deck_0,bathroom_2,deck_1,deck_0,stairs,kitchen,deck_0,bathroom_3,kitchen,stairs,kitchen,deck_brush,bathroom_2
Delwin,stairs,deck_1,deck_0,kitchen,bathroom_3,kitchen,deck_brush,deck_0,bathroom_2,deck_1,stairs,kitchen,deck_0,kitchen,bathroom_2,bathroom_3,stairs
Dylan,kitchen,bathroom_3,deck_0,stairs,bathroom_2,kitchen,bathroom_2,deck_brush,deck_1,deck_0,deck_0,deck_brush,kitchen,stairs,deck_1,bathroom_3,kitchen
Ian,bathroom_2,deck_brush,kitchen,bathroom_3,kitchen,deck_0,stairs,deck_0,kitchen,bathroom_2,deck_1,deck_0,bathroom_3,kitchen,deck_brush,deck_1,stairs


In [11]:
weekly_counts = []

for week, assignment in weekly_assignments.items():
    counts = {name: {c: 0 for c in cleanup_types} for name in names}
    for person, cleanup in assignment.items():
        counts[person][cleanup] += 1
    week_df = pd.DataFrame.from_dict(counts, orient='index')
    week_df['week'] = week
    weekly_counts.append(week_df)

weekly_counts_df = pd.concat(weekly_counts)
weekly_counts_df

,deck_1,kitchen,deck_0,deck_brush,bathroom_2,bathroom_3,stairs,week
Andre,0,0,1,0,0,0,0,1
Austin,1,0,0,0,0,0,0,1
Bhuvan,1,0,0,0,0,0,0,1
Christian,0,0,0,1,0,0,0,1
Clement,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...
Simon,0,0,1,0,0,0,0,17
Toney,0,0,0,0,1,0,0,17
Vivek,0,0,0,1,0,0,0,17
Yash,0,1,0,0,0,0,0,17
